In [1]:
# ref: https://www.kaggle.com/jacklinggu/lstm-with-glove-embedding-public-lb-score-0-049

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from models_def import cnn2d,cnn_v1,cnn_v2,cudnn_gru,lstm_v1,cnn_gru,gru_v1
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import text, sequence

model_defines = [
#     [cnn2d,'cnn2d'],
#     [cnn_v1,'cnn_v1'],
#     [cnn_v2,'cnn_v2'],
    [cudnn_gru,'cudnn_gru'],
    [lstm_v1,'lstm_v1'],
    [cnn_gru,'cnn_gru'],
    [gru_v1,'gru_v1']
]

max_features = 40000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

Using TensorFlow backend.


(159571, 6)


In [2]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)
EMBEDDING_DIM = 300
nb_words = max_features

(159571, 150) (153164, 150)


In [ ]:
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import KFold
from keras import backend as K
import pickle
from keras.models import load_model
import gc

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def kf_train(model_func,fold_cnt=3,rnd=1,epo=10,batch=64):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = model_func(maxlen,nb_words,EMBEDDING_DIM,embedding_matrix=None,trainable_flag=True,comp=True)
        batch_size = batch
        epochs = epo
        file_path="best_model.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
        
        # clear
        del model
        gc.collect()
        K.clear_session()
        
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    eval_val(y,train_pred)
    return train_pred, test_pred

rnd_init = 1
for model_func,model_name in model_defines:
    for fold_k in [5]:
        print('==================================')
        epo = 6
        batch = 64
        train_pred,test_pred = kf_train(model_func,fold_k,rnd_init,epo,batch)
        output_feat = '../features/no_pretrained_{}_{}_feat.pkl'.format(model_name,fold_k)
        with open(output_feat,'wb') as fout:
            pickle.dump([train_pred,test_pred],fout)
        print(output_feat,'done')
        rnd_init += 1
        




Train on 127656 samples, validate on 31915 samples
Epoch 1/6
127656/127656 [==============================] - 166s 1ms/step - loss: 0.0600 - acc: 0.9793 - val_loss: 0.0487 - val_acc: 0.9820
Epoch 2/6
127656/127656 [==============================] - 168s 1ms/step - loss: 0.0444 - acc: 0.9832 - val_loss: 0.0492 - val_acc: 0.9821
Epoch 3/6
127656/127656 [==============================] - 171s 1ms/step - loss: 0.0371 - acc: 0.9854 - val_loss: 0.0537 - val_acc: 0.9818
Epoch 4/6
127656/127656 [==============================] - 173s 1ms/step - loss: 0.0324 - acc: 0.9870 - val_loss: 0.0582 - val_acc: 0.9819
Epoch 5/6
127656/127656 [==============================] - 173s 1ms/step - loss: 0.0291 - acc: 0.9883 - val_loss: 0.0609 - val_acc: 0.9809
Epoch 6/6
127656/127656 [==============================] - 173s 1ms/step - loss: 0.0263 - acc: 0.9895 - val_loss: 0.0640 - val_acc: 0.9814
Train on 127657 samples, validate on 31914 samples
Epoch 1/6
127657/127657 [==============================] - 247s 

Epoch 6/6
127657/127657 [==============================] - 164s 1ms/step - loss: 0.0268 - acc: 0.9893 - val_loss: 0.0696 - val_acc: 0.9808
-------------------------------
0 0.106547423812 0.960393805892
1 0.0237582529162 0.990154852699
2 0.0523350104538 0.980478909075
3 0.0121699123789 0.997060869456
4 0.0676647065367 0.973084081694
5 0.0281385979614 0.991533549329
final 0.0484356506765 0.982117678024
../features/no_pretrained_cnn2d_5_feat.pkl done
Train on 127656 samples, validate on 31915 samples
Epoch 1/6
127656/127656 [==============================] - 93s 726us/step - loss: 0.0625 - acc: 0.9787 - val_loss: 0.0496 - val_acc: 0.9814
Epoch 2/6
127656/127656 [==============================] - 92s 722us/step - loss: 0.0454 - acc: 0.9827 - val_loss: 0.0492 - val_acc: 0.9818
Epoch 3/6
127656/127656 [==============================] - 92s 718us/step - loss: 0.0390 - acc: 0.9846 - val_loss: 0.0512 - val_acc: 0.9813
Epoch 4/6
127656/127656 [==============================] - 92s 718us/step - 

127657/127657 [==============================] - 92s 719us/step - loss: 0.0390 - acc: 0.9847 - val_loss: 0.0513 - val_acc: 0.9821
Epoch 4/6
127657/127657 [==============================] - 92s 718us/step - loss: 0.0344 - acc: 0.9862 - val_loss: 0.0557 - val_acc: 0.9817
Epoch 5/6
127657/127657 [==============================] - 92s 718us/step - loss: 0.0307 - acc: 0.9875 - val_loss: 0.0636 - val_acc: 0.9810
Epoch 6/6
127657/127657 [==============================] - 92s 718us/step - loss: 0.0284 - acc: 0.9885 - val_loss: 0.0674 - val_acc: 0.9805
-------------------------------
0 0.107626946912 0.961214757067
1 0.0239273013294 0.990098451473
2 0.0524333514011 0.980623045541
3 0.0115861175448 0.997092203471
4 0.0690156564816 0.972570203859
5 0.0266089366595 0.991633818175
final 0.0485330517214 0.982205413264
../features/no_pretrained_cnn_v1_5_feat.pkl done
Train on 127656 samples, validate on 31915 samples
Epoch 1/6
127656/127656 [==============================] - 113s 885us/step - loss: 0

Train on 127657 samples, validate on 31914 samples
Epoch 1/6
127657/127657 [==============================] - 110s 864us/step - loss: 0.0605 - acc: 0.9791 - val_loss: 0.0507 - val_acc: 0.9815
Epoch 2/6
127657/127657 [==============================] - 113s 886us/step - loss: 0.0447 - acc: 0.9829 - val_loss: 0.0495 - val_acc: 0.9821
Epoch 3/6
127657/127657 [==============================] - 111s 870us/step - loss: 0.0369 - acc: 0.9852 - val_loss: 0.0553 - val_acc: 0.9815
Epoch 4/6
127657/127657 [==============================] - 112s 875us/step - loss: 0.0325 - acc: 0.9868 - val_loss: 0.0595 - val_acc: 0.9815
Epoch 5/6
127657/127657 [==============================] - 112s 878us/step - loss: 0.0292 - acc: 0.9883 - val_loss: 0.0609 - val_acc: 0.9799
Epoch 6/6
127657/127657 [==============================] - 112s 879us/step - loss: 0.0261 - acc: 0.9895 - val_loss: 0.0687 - val_acc: 0.9789
-------------------------------
0 0.110172104541 0.959391117434
1 0.0246386042259 0.990399258011
2 0.05